# XGBoost

Ranklib is a relatively old library and doesn't have the wide spread use that XGBoost does. Ranklib is still under active development, but the fork of the project OSC created reflects an older version.

The ES-LTR plugin is designed to work with XGBoost model format. This notebook starts with the `classic` training data generated in `hello-ltr.py` and shows how you could use XGBoost instead of Ranklib to create a model and use it with the plugin.

### Input Data

Gather the data generated for our `classic` model in `hello-ltr.ipynb`. If this file doesn't exist yet, rerun that notebook!

In [1]:
import ltr.judgments as judge
df = [j for j in judge.judgments_from_file(open('data/classic-training.txt'))]
df = judge.judgments_to_dataframe(df)
df

Recognizing 1 queries


/Users/markcoh/IdeaProjects/OpenSearchWork/hello-ltr/ltr/judgments.py:266: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only



,features0,uid,qid,keywords,docId,grade
0,2014.0,1_374430,1,,374430,0
1,1995.0,1_19404,1,,19404,1
2,1994.0,1_278,1,,278,1
3,2016.0,1_372058,1,,372058,0
4,1972.0,1_238,1,,238,2
...,...,...,...,...,...,...
995,1960.0,1_967,1,,967,3
996,2007.0,1_13008,1,,13008,1
997,1999.0,1_4951,1,,4951,1
998,2015.0,1_228205,1,,228205,0


### Libraries for xgboost-ing

Just the dependencies we need to train and visualize out model trained with XG-Boost instead of Ranklib.

In [2]:
import pandas as pd
import xgboost as xgb
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 50,150

### Set up our training Matrix

XGBoost has it's data specficiations so we need to get out features into that format to use it.


In [3]:
df = df[['grade', 'features0']]
features = df[['features0']]
labels = df[['grade']]

dmx = xgb.DMatrix(features, labels)

### Train the first XGBoost model

Using the demo parameters for our model, we will train a standard regression tree

In [4]:
param = {'max_depth':2, 'eta':1, 'silent':1}
num_round = 2

model = xgb.train(param, dmx, num_round)

[22:52:03] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




### Inspect as dataframe

Looking at the model as a dataframe can tell you which splits helped the most

In [5]:
model.trees_to_dataframe()

,Tree,Node,ID,Feature,Split,Yes,No,Missing,Gain,Cover,Category
0,0,0,0-0,features0,1990.5,0-1,0-2,0-1,1087.860720,1000.0,NaN
1,0,1,0-1,features0,1970.5,0-3,0-4,0-3,178.059326,414.0,NaN
2,0,2,0-2,features0,2010.5,0-5,0-6,0-5,136.161972,586.0,NaN
3,0,3,0-3,Leaf,NaN,NaN,NaN,NaN,2.828326,232.0,NaN
4,0,4,0-4,Leaf,NaN,NaN,NaN,NaN,1.491803,182.0,NaN
5,0,5,0-5,Leaf,NaN,NaN,NaN,NaN,0.498649,369.0,NaN
6,0,6,0-6,Leaf,NaN,NaN,NaN,NaN,-0.497706,217.0,NaN
7,1,0,1-0,features0,1950.5,1-1,1-2,1-1,37.753029,1000.0,NaN
8,1,1,1-1,Leaf,NaN,NaN,NaN,NaN,0.663278,79.0,NaN
9,1,2,1-2,features0,1970.5,1-3,1-4,1-3,13.812227,921.0,NaN


In [ ]:
xgb.plot_tree(model)

### Adjust the objective for LTR

Really we don't want the regression as our objective function. In LTR we take advantage of a new pairwise loss function to find the optimal splits for a regression tree. 

This doesn't make a massive difference for the model that is generated because it is still a regression tree at the end of the day, but we are not longer using residual sqared error.

In [ ]:
param2 = {'max_depth':2, 'eta':1, 'silent':1, 'objective':'rank:pairwise'}

ranking_model = xgb.train(param2, dmx, num_round)

In [ ]:
ranking_model.trees_to_dataframe()

In [ ]:
xgb.plot_tree(ranking_model)

### Uploading an XGBoost model to the plugin

Since the model can be represented with JSON, the plugin can parse it. But we need to make sure the plugin gets the proper feature value names in order for it to parse properly.

These are supplied via a mapping `txt` file, `fmap.txt`.

The first step is to dump the model with the feature mapping to the features already stored in the plugin.

In [ ]:
model_dump = ranking_model.get_dump(fmap='fmap.txt', dump_format='json')

### Massage the JSON

Manipulate the XGBoost output format to clean it up for posting to the plugin.

In [ ]:
import json
clean_model = []
for line in model_dump:
    clean_model.append(json.loads(line))

### Post it to the plugin

Still referencing the index and feature set the model will be associated with.

In [ ]:
import ltr.client as client
client = client.ElasticClient()

client.submit_xgboost_model('release', 'tmdb', 'xgb', clean_model)

### Confirm it works

In [ ]:
from ltr.release_date_plot import search
search(client, 'batman', 'xgb')

### Compare it to the classic Ranklib model

In [ ]:
from ltr.release_date_plot import plot
plot(client, "batman", models = ['classic', 'xgb'])